# NJIT Fall 2024 CS634: Data Mining Final Project #
## Joshua Kobuskie ##
### November 17, 2024 ###

##### Supervised Data Mining (Classification) Binary Classification Only #####

Implement 3 different classification algorithms in Python. One of them is Random
Forest , the second one is from the deep learning list in the “Appendix → Additional
Option: Deep Learning”, and the third is from the list of algorithms in “Appendix →
Additional Option: Algorithms” on 1 dataset of your choice (each of the three
algorithms must run on the same dataset).

NOTE: This is not from scratch implementation, just use the existing libraries
to implement the algorithms, but the performance metrics must be calculated
manually. You may use “confusion_matrix” library to get TP, TN, FP, FN ONLY,
then calculate the FPR, FNR, etc.… using the formulas from the slides.

Sources of data are listed in the Appendix “Additional Option: Sources of Data” or use
your own.

Your final term project documentation must clearly indicate the algorithms and dataset
you used in the project.

The classification algorithms that I have chosen to implement and evaluate in this project are the Random Forest, the UNDECIDED, and the UNDECIDED.<br>
The dataset I have chosen to use in this project is Kddcup 99 dataset from the scikit-learn real world datasets. This dataset contains TCP network connection data with about 5 million training records and 2 million test records. Each record is labelled as normal, or as a specific attack type. To ensure that this data will meet the criteria for binary classification used in this project, the data will be seperated into 2 classes of either normal or attack, rather than the specific classes of attacks detailed within the dataset. This also mimicks the intention of the dataset, as the training data contains 24 types of attack, but the test data contains an additonal 14 types of attacks not previously seen in the training data. This will test the models ability to detect known and unknown attacks based on its understnading of normal records. The models I will implment will be able to classify records as either normal TCP network connection data or abnormal TCP network connection data indicative of an attack.

In addition to the general submission rules and grading, include the websites where the
software and complete dataset can be downloaded.

The scikit-learn software can be downloaded at the following link: https://scikit-learn.org/stable/install.html <br>
The Kddcup 99 dataset can be downloaded in its entirety at the following link: https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [99]:
from sklearn.datasets import fetch_kddcup99
import numpy as np
import pandas as pd
dataset = fetch_kddcup99(as_frame=True).frame
for col in dataset.iloc[0]:
    print(type(col))

<class 'int'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'bytes'>


This dataset contains all the data needed for classification, but currently stores all of the protocol_type, service, flag, and labels as bytes. These values will be encoded into ints for training and testing. This data also presents a multiclassification problem. To transform this into a binary classification problem, we will have two class labels of normal and attack. All non-normal labels will be considered an attack for training and testing purposes.

In [100]:
def transformEncode(col, dict, count):
    if col not in dict:
        dict[col] = count[0]
        dict[count[0]] = col
        count[0] += 1
    return dict[col]

def transformLabel(col):
    if col == b'normal.':
        return 0
    else:
        return 1


protocolDict = {}
#Must be a list so that a reference is passed and we can change the value outside the function
count = [0]
dataset["protocol_type"] = dataset["protocol_type"].apply(transformEncode, args=(protocolDict,count,))

serviceDict = {}
count = [0]
dataset["service"] = dataset["service"].apply(transformEncode, args=(serviceDict,count,))

flagDict = {}
count = [0]
dataset["flag"] = dataset["flag"].apply(transformEncode, args=(flagDict,count,))

#Normal will be 0, attack will be 1
labelDict = {b'normal':0, 0: b'normal', b'attack':1, 1:b'attack'}
dataset["labels"] = dataset["labels"].apply(transformLabel)

print(protocolDict)

{b'tcp': 0, 0: b'tcp', b'udp': 1, 1: b'udp', b'icmp': 2, 2: b'icmp'}
